In [7]:
import requests
from bs4 import BeautifulSoup
import re

# 1. نرسل طلب إلى الموقع
url = "https://swag.sa/?srsltid=AfmBOopDZbqWR_2BfrVS288CafizRQXvftPsnn5LQMM-npQ2JJmS9tJk"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
                  " AppleWebKit/537.36 (KHTML, like Gecko)"
                  " Chrome/120.0.0.0 Safari/537.36",
    "Accept-Language": "ar-SA,ar;q=0.9,en;q=0.8"
}
response = requests.get(url, headers=headers, timeout=20)

# 2. التحقق من نجاح الطلب
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')

    # 3. نحاول العثور على عناصر المنتجات (عدة محاولات احتياطية للـ selectors)
    selectors = [
        "div.product", "div.product-item", "li.product", "div.product-card",
        ".product", ".product-item", ".product-card", "article.product"
    ]

    items = []
    for sel in selectors:
        found = soup.select(sel)
        if found:
            items = found
            break

    # لو ما وجدنا شيء، نبحث عن عناصر قد تحتوي على سعر كنقطة انطلاق
    if not items:
        all_tags = soup.find_all(True)
        price_re = re.compile(r"[\d\.,]+\s*(?:SAR|ر\.س|ريال|\$|دولار)?", re.I)
        potential = []
        for t in all_tags:
            txt = t.get_text(" ", strip=True)
            if price_re.search(txt) and len(txt) < 400:
                potential.append(t)
        items = potential[:200]

    results = []
    price_pattern = re.compile(r"[\d\.,]+(?:\s*(?:SAR|ر\.س|ريال|\$|دولار))?", re.I)

    for el in items:
        # استخراج اسم المنتج بطرق متعددة (أبسط الطرق فقط)
        title = None
        if el.get("data-title"):
            title = el.get("data-title")
        if not title:
            a = el.select_one("a[title]")
            if a:
                title = a.get("title")
        if not title:
            img = el.find("img")
            if img and img.get("alt"):
                title = img.get("alt")
        if not title:
            h = el.find(["h1","h2","h3","h4"])
            if h and h.get_text(strip=True):
                title = h.get_text(strip=True)
        if not title:
            ttag = el.select_one(".product-name, .name, .title")
            if ttag:
                title = ttag.get_text(" ", strip=True)

        # استخراج السعر بطرق بسيطة
        price = None
        for cls in [".price", ".product-price", ".price-current", ".amount", ".price-tag"]:
            p = el.select_one(cls)
            if p and p.get_text(strip=True):
                price = p.get_text(" ", strip=True)
                break
        if not price:
            # البحث النصي داخل العنصر عن نمط سعر
            txt = el.get_text(" ", strip=True)
            m = price_pattern.search(txt)
            if m:
                price = m.group(0)

        # تنظيف
        if title:
            title = re.sub(r"\s+", " ", title).strip()
        if price:
            price = price.replace("\n", " ").strip()

        if title or price:
            results.append({"title": title or "", "price": price or ""})

    # طباعة جدول بسيط مثل الكود الأصلي
    if not results:
        print("لم يتم العثور على منتجات أو أسعار ضمن الـ HTML المستلم. الاحتمال الأكبر أن الصفحة تُحمَّل عبر JavaScript.")
    else:
        # إزالة مكررات
        seen = set()
        cleaned = []
        for r in results:
            key = (r["title"].strip().lower(), r["price"].strip())
            if key not in seen:
                seen.add(key)
                cleaned.append(r)
        # طباعة بيانات
        print("قائمة المنتجات وأسعارها:\n")
        for r in cleaned:
            print(f"- {r['title']} | السعر: {r['price']}")
else:
    print("فشل تحميل الصفحة. رمز الاستجابة:", response.status_code)


قائمة المنتجات وأسعارها:

-  | السعر: 249 ريال
-  | السعر: 5
-  | السعر: 15
- SWAG | السعر: 0
- user | السعر: 0
-  | السعر: 0
- اختر 5 قطع ب 195 ريال | السعر: 5
- تي شيرت أوفر سايز ساده أنيق وعصري | السعر: 5
-  | السعر: 3
- تي شيرت أوفر سايز ساده أنيق وعصري | السعر: 195.00  ر.س
-  | السعر: 195.00  ر.س
- تي شيرت شبابي بتخريمات جانبية مميزة | السعر: 5
-  | السعر: 1
- تي شيرت شبابي بتخريمات جانبية مميزة | السعر: 195.00  ر.س
- تي شيرت أوفر سايز ساده بألوان متعددة | السعر: 5
- تي شيرت أوفر سايز ساده بألوان متعددة | السعر: 195.00  ر.س
- تي شيرت مزين بكتابة اماميه | السعر: 5
-  | السعر: 2
- تي شيرت مزين بكتابة اماميه | السعر: 195.00  ر.س
- تي شيرت مزين بكتابات أنيقه | السعر: 5
- تي شيرت مزين بكتابات أنيقه | السعر: 195.00  ر.س
- تي شيرت بطبعات مميزة وأنيقة | السعر: 195.00  ر.س
- شورت كارجو جينز بجيوب كبيرة | السعر: 5
- شورت كارجو جينز بجيوب كبيرة | السعر: 195.00  ر.س
- تي شيرت شبابي بألوان صيفية | السعر: 5
- تي شيرت شبابي بألوان صيفية | السعر: 195.00  ر.س
- بنطلون قطن كارجو بمظهر عصري ومريح | 